
#### Introduction

Implementation of RNNs for sentence generation using Keras. The reference taken is [this](https://chunml.github.io/ChunML.github.io/project/Creating-Text-Generator-Using-Recurrent-Neural-Network/) website. The dataset use is Andrej Karpathy's [page](https://cs.stanford.edu/people/karpathy/char-rnn/)

First we will download the text if required and load the data as characters in the text

In [17]:
import os
from urllib.request import urlretrieve

def maybe_download(target_url, local_dir, local_file):
    if not os.path.exists(local_dir):
        os.mkdir(local_dir)
        
    complete_path = os.path.join(local_dir, local_file)
    if os.path.exists(complete_path):
        print('File %s exists, not downloading'%complete_path)
    else:
        print('Downloading %s'%target_url)
        urlretrieve(target_url, complete_path)
        print('File downloaded')
    return complete_path

url = 'https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt'
local_file = maybe_download(url, 'Shakespeare', 'shakespeare_input.txt')

with open(local_file, 'r') as f:
    data = f.read()
    
chars = list(set(data))
print('Number of unique characters are', len(chars))

File Shakespeare/shakespeare_input.txt exists, not downloading
Number of unique characters are 67


Create lookup and reverse lookup for vocabulary

In [16]:
char_to_ix = {c:ix for ix, c in enumerate(chars)}
ix_to_char = {ix:c for ix, c in enumerate(chars)}

Lets define some parameters for the network

In [19]:

seq_length = 50 #Number of previous characters (timesteps) to train on

vocab_size = len(chars)



Define input and output 